In [1]:
# Фазификация
import sqlite3
conn = sqlite3.connect('lab3.db', check_same_thread=False)
cursor = conn.cursor()
#Linguistic_Var1 = input() # 0,25 - Объем кружки
#Linguistic_Var2 = input() # 0,2 - Сладость
Linguistic_Var1 = '0.25' # 0,25 - Объем кружки
Linguistic_Var2 = '0.2' # 0,2 - Сладость
cursor.execute("SELECT name, ch1_name, ch1_val, ch2_name, ch2_val, ch3_name, ch3_val FROM FuzzyVariables where linguistic_variable_id=? and ling_var_val=?",(1,Linguistic_Var1,))
variable_1 = cursor.fetchone()
cursor.execute("SELECT name, ch1_name, ch1_val, ch2_name, ch2_val, ch3_name, ch3_val FROM FuzzyVariables where linguistic_variable_id=? and ling_var_val=?",(2,Linguistic_Var2,))
variable_2 = cursor.fetchone()
print('Var1 = ', variable_1)
print('Var2 = ', variable_2)

Var1 =  ('Объем кружки', 'малая', 1, 'средняя', 0, 'большая', 0)
Var2 =  ('Сладость', 'не сладко', 0.5, 'сладко', 0.6, 'очень сладко', 0)


In [2]:
# Агрегирование
cursor.execute("SELECT ch1_val, ch2_val, ch3_val FROM FuzzyVariables where linguistic_variable_id=? and ling_var_val=?",(1,Linguistic_Var1,))
variable_1 = cursor.fetchone()
cursor.execute("SELECT ch1_val, ch2_val, ch3_val FROM FuzzyVariables where linguistic_variable_id=? and ling_var_val=?",(2,Linguistic_Var2,))
variable_2 = cursor.fetchone()
aggr_func = []
aggr_func.append(min(variable_1[0],variable_2[0])) # мало
aggr_func.append(min(variable_1[0],variable_2[1])) # средне
aggr_func.append(min(variable_1[0],variable_2[2])) # средне
aggr_func.append(min(variable_1[1],variable_2[0])) # мало
aggr_func.append(min(variable_1[1],variable_2[1])) # средне
aggr_func.append(min(variable_1[1],variable_2[2])) # мало
aggr_func.append(min(variable_1[2],variable_2[0])) # средне 
aggr_func.append(min(variable_1[2],variable_2[1])) # много 
aggr_func.append(min(variable_1[2],variable_2[2])) # много
aggr_func

[0.5, 0.6, 0, 0, 0, 0, 0, 0, 0]

In [7]:
# Активация
act_func = []
act_func.append(max(aggr_func[0],aggr_func[3],aggr_func[5]))
act_func.append(max(aggr_func[1],aggr_func[2],aggr_func[4],aggr_func[6]))
act_func.append(max(aggr_func[7],aggr_func[8]))
cursor.execute("SELECT COUNT(*) FROM FuzzyVariables where linguistic_variable_id=3")
counter_act = cursor.fetchone()
counter = counter_act[0]
for i in range(counter):
    x = 13+i
    cursor.execute("SELECT * FROM FuzzyVariables where linguistic_variable_id=3 and id=?",(x,))
    variable_3 = cursor.fetchone()
    if act_func[0] < variable_3[5]:
        ch1_val = act_func[0]
        cursor.execute("Insert into Func_Activation(name,linguistic_variable_id,ling_var_val,ch1_name,ch1_val,ch2_name,ch2_val,ch3_name,ch3_val) VALUES(?,?,?,?,?,?,?,?,?)",(variable_3[1],variable_3[2],variable_3[3],variable_3[4],ch1_val,variable_3[6],variable_3[7],variable_3[8],variable_3[9]))
        conn.commit()
    elif act_func[1] < variable_3[7]:
        ch2_val = act_func[1]
        cursor.execute("Insert into Func_Activation(name,linguistic_variable_id,ling_var_val,ch1_name,ch1_val,ch2_name,ch2_val,ch3_name,ch3_val) VALUES(?,?,?,?,?,?,?,?,?)",(variable_3[1],variable_3[2],variable_3[3],variable_3[4],variable_3[5],variable_3[6],ch2_val,variable_3[8],variable_3[9]))
        conn.commit()
    elif act_func[2] < variable_3[9]:
        ch3_val = act_func[2]
        cursor.execute("Insert into Func_Activation(name,linguistic_variable_id,ling_var_val,ch1_name,ch1_val,ch2_name,ch2_val,ch3_name,ch3_val) VALUES(?,?,?,?,?,?,?,?,?)",(variable_3[1],variable_3[2],variable_3[3],variable_3[4],variable_3[5],variable_3[6],variable_3[7],variable_3[8],ch3_val))
        conn.commit()
    else:
        cursor.execute("Insert into Func_Activation(name,linguistic_variable_id,ling_var_val,ch1_name,ch1_val,ch2_name,ch2_val,ch3_name,ch3_val) VALUES(?,?,?,?,?,?,?,?,?)",(variable_3[1],variable_3[2],variable_3[3],variable_3[4],variable_3[5],variable_3[6],variable_3[7],variable_3[8],variable_3[9]))
        conn.commit()

In [8]:
act_func

[0.5, 0.6, 0]

In [4]:
# Аккумуляция
accum_max = []
for i in range(counter):
    i += 1
    cursor.execute("SELECT ch1_val, ch2_val, ch3_val FROM Func_Activation where id=?",(i,))
    accum = cursor.fetchone()
    accum_max.append(max(accum))
accum_max

[0.5, 0.5, 0.4, 0.6, 0.4, 0.2, 0]

In [5]:
# Дефазификация 
# Формула x(i) * y(i) / y(0-i)
defas = []
sum_defas = 0
sum_accum_max = 0
result = 0
x = 0
for i in range(counter):
    x += 1
    cursor.execute("SELECT ling_var_val FROM Func_Activation where id=?",(x,))
    func_def = cursor.fetchone()
    def_sum = func_def[0]
    defas.append(def_sum * accum_max[i])
# Сумма всех элементов, полученных умножением  
for c in range(len(defas)):
    sum_defas += defas[c] 
# Сумма всех элементов, полученных на этапе аккумуляции
for b in range(len(accum_max)):
    sum_accum_max += accum_max[b]
# Деление
print('{} -> {}'.format(defas,sum_defas))
print('{} -> {}'.format(accum_max,sum_accum_max))
result = sum_defas / sum_accum_max
result

[0.0, 0.25, 0.4, 0.8999999999999999, 0.8, 0.5, 0] -> 2.8499999999999996
[0.5, 0.5, 0.4, 0.6, 0.4, 0.2, 0] -> 2.6


1.096153846153846

In [9]:
#cursor.execute("DELETE FROM Func_Activation") # TRUNCATE TABLE
#cursor.execute("DELETE FROM sqlite_sequence where name='Func_Activation'") # AUTO-INC-RESET
#conn.commit()